## Registering our Model
### Getting our best runs

In [1]:
import mlflow
from mlflow.tracking import MlflowClient
import os

In [2]:
#Get experiment
mlflow.set_tracking_uri("sqlite:///mlflow.db")
experiment = mlflow.get_experiment_by_name("elastic_net_experiment")
experiment

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='elastic_net_experiment', tags={}>

In [3]:
# Get experiment runs
client = MlflowClient()
runs = client.search_runs(experiment_ids = [experiment.experiment_id])
runs

[<Run: data=<RunData: metrics={'mae': 51.05182860408632, 'r2': 0.3951809598912357, 'rmse': 63.24666777876805}, params={'alpha': '0.01', 'l1_ratio': '1.0'}, tags={'mlflow.log-model.history': '[{"run_id": "a365265825ca440fa9aaa9eaf4efe603", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2023-01-22 14:38:29.136398", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"loader_module": "mlflow.sklearn", '
                              '"python_version": "3.7.6", "env": "conda.yaml"}, '
                              '"sklearn": {"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.0.2", '
                              '"serialization_format": "cloudpickle", "code": '
                              'null}}, "model_uuid": '
                              '"3ec35082b99e473a87ce0c2ea2eb7dd9", '
                              '"mlf

In [4]:
#Get the best metrics value and use it to find the best run

metrics = [run.data.metrics["rmse"] for run in runs]
best_rmse = min(metrics)
best_run = [run for run in runs if run.data.metrics["rmse"] == best_rmse][0]
print(best_rmse)
best_run.info

63.24666777876805


<RunInfo: artifact_uri='./mlruns/1/a365265825ca440fa9aaa9eaf4efe603/artifacts', end_time=1674398312129, experiment_id='1', lifecycle_stage='active', run_id='a365265825ca440fa9aaa9eaf4efe603', run_uuid='a365265825ca440fa9aaa9eaf4efe603', start_time=1674398308346, status='FINISHED', user_id='visalakshi'>

In [6]:
# Register the model from the best run and log it

model_name = "my_enet_model"
mlflow.set_experiment(experiment_name = "elastic_net_experiment")
model_uri = f"runs:/{best_run.info.run_id}/model"
model_version = mlflow.register_model(model_uri = model_uri, name = model_name)
model_version

Registered model 'my_enet_model' already exists. Creating a new version of this model...
2023/01/22 20:12:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: my_enet_model, version 2
Created version '2' of model 'my_enet_model'.


<ModelVersion: creation_timestamp=1674398575614, current_stage='None', description=None, last_updated_timestamp=1674398575614, name='my_enet_model', run_id='a365265825ca440fa9aaa9eaf4efe603', run_link=None, source='./mlruns/1/a365265825ca440fa9aaa9eaf4efe603/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

## Loading our model

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
model = client.get_registered_model(name = "my_enet_model")
model

In [ ]:
model_artifact = mlflow.sklearn.load_model(model_uri)
model_artifact

## Running inference

In [ ]:
from sklearn import datasets
import numpy as np

diabetes = datasets.load_diabetes()
data = diabetes.data
pred = model_artifact.predict(data)
pred

## Adding metadata to our model

In [ ]:
client.set_model_version_tag(model_name, model_version.version, key = "prediction_works", value = "true")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)
model_version_info.tags

## Promoting our model to Staging

We are happy with our model (for now), so let's promote it to Staging

In [9]:
client.transition_model_version_stage(model_name, model_version.version, stage = "Production")

<ModelVersion: creation_timestamp=1674398575614, current_stage='Production', description=None, last_updated_timestamp=1674398727383, name='my_enet_model', run_id='a365265825ca440fa9aaa9eaf4efe603', run_link=None, source='./mlruns/1/a365265825ca440fa9aaa9eaf4efe603/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [8]:
model_version_info = client.get_model_version(model_name, model_version.version)
model_version_info.current_stage

'Staging'